# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234740752913                   -0.50    7.0
  2   -7.249778240275       -1.82       -1.39    1.0
  3   -7.251179348323       -2.85       -1.96    2.0
  4   -7.251251864259       -4.14       -2.15    2.0
  5   -7.251300702870       -4.31       -2.37    1.0
  6   -7.251336267140       -4.45       -2.92    1.0
  7   -7.251338693644       -5.62       -3.62    1.0
  8   -7.251338777016       -7.08       -3.85    2.0
  9   -7.251338796693       -7.71       -4.43    1.0
 10   -7.251338798611       -8.72       -5.04    2.0
 11   -7.251338798684      -10.13       -5.51    1.0
 12   -7.251338798701      -10.79       -5.72    3.0
 13   -7.251338798703      -11.58       -5.99    1.0
 14   -7.251338798704      -12.01       -6.49    1.0
 15   -7.251338798705      -12.62       -6.98    3.0
 16   -7.251338798705      -13.97       -7.25    2.0
 17   -7.251338798705      -14.45       -7.60 

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05299583454794791
[ Info: Arnoldi iteration step 2: normres = 0.5125137114651308
[ Info: Arnoldi iteration step 3: normres = 0.9372774701903317
[ Info: Arnoldi iteration step 4: normres = 0.2777072774055313
[ Info: Arnoldi iteration step 5: normres = 0.5477894434711637
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.96e-02, 8.18e-02, 4.90e-01, 2.27e-01, 1.76e-02)
[ Info: Arnoldi iteration step 6: normres = 0.2105942091751543
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (7.09e-03, 1.29e-01, 1.15e-01, 9.55e-02, 1.63e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09212410901742929
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.07e-04, 9.97e-03, 8.87e-03, 4.14e-02, 7.97e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10767950018746926
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.41e-0